In [ ]:
# Sistema de Almacenamiento Distribuido

Este cuaderno explica el código de un sistema de almacenamiento distribuido, que incluye cifrado y replicación de datos. A continuación, se detalla cada bloque de código y su funcionalidad.

## Importar Librerías Necesarias

Primero, importamos las librerías necesarias para la aplicación.

```python
from flask import Flask, request, send_file, jsonify
from Crypto.Cipher import AES
import os
import logging
import asyncio
import aiohttp
import threading
from queue import Queue



- **Flask:** Microframework para construir aplicaciones web, permitiendo manejar solicitudes HTTP.
- **Crypto.Cipher.AES:** Librería para el cifrado de datos utilizando el estándar AES (Advanced Encryption Standard), que es un algoritmo de cifrado simétrico ampliamente utilizado.
- **os:** Módulo para interactuar con el sistema operativo, permitiendo manipular archivos y directorios.
- **logging:** Módulo para generar registros de eventos, útil para el diagnóstico y solución de problemas.
- **asyncio y aiohttp:** Librerías para manejar operaciones asíncronas, permitiendo realizar múltiples tareas de forma concurrente sin bloquear el flujo del programa.
- **threading y Queue:** Módulos para manejar operaciones concurrentes utilizando hilos y colas, lo que permite ejecutar varias operaciones al mismo tiempo.

**Configuración Inicial**

Configuramos las constantes y creamos las carpetas necesarias.

In [ ]:
UPLOAD_FOLDER = '/app/cargas'
key = b'This_is_a16b_key'
NODOS = ["http://storage-node-1:5000", "http://storage-node-2:5000"]

os.makedirs(UPLOAD_FOLDER, exist_ok=True)
logging.basicConfig(level=logging.DEBUG)


- UPLOAD_FOLDER: Directorio donde se almacenarán los archivos.
- key: Clave de cifrado AES.
- NODOS: Lista de nodos donde se replicarán los archivos.
- os.makedirs: Crea el directorio si no existe.
- logging.basicConfig: Configura el nivel de registro a DEBUG.

**Función de Cifrado**

Función para cifrar datos utilizando AES.

In [ ]:
def cifrar_archivo(data):
    cipher = AES.new(key, AES.MODE_EAX)
    ciphertext, tag = cipher.encrypt_and_digest(data)
    return cipher.nonce, tag, ciphertext


- cifrar_archivo: Cifra los datos utilizando la clave AES, y devuelve el nonce, el tag y el texto cifrado.

**Función de Descifrado**

Función para descifrar datos utilizando AES.

In [ ]:
def descifrar_archivo(nonce, tag, ciphertext):
    cipher = AES.new(key, AES.MODE_EAX, nonce=nonce)
    data = cipher.decrypt_and_verify(ciphertext, tag)
    return data


- descifrar_archivo: Descifra los datos utilizando la clave AES y el nonce, y devuelve el texto original.

**Función Asíncrona para Replicar Archivos**

Función para replicar archivos a otros nodos de manera asíncrona.

In [ ]:
async def replicar_archivo_async(nodo, ruta_archivo, nonce, tag):
    try:
        form = aiohttp.FormData()
        with open(ruta_archivo, 'rb') as file_enc:
            data = file_enc.read()
        form.add_field('archivo', data, filename=os.path.basename(ruta_archivo))
        form.add_field('nonce', nonce.hex())
        form.add_field('tag', tag.hex())
        async with aiohttp.ClientSession() as session:
            async with session.post(f"{nodo}/cargar", data=form) as response:
                if response.status != 200:
                    logging.error(f"Error replicando en {nodo}: {response.status}")
                else:
                    logging.debug(f"Replicación en {nodo} completada con estado {response.status}")
    except Exception as e:
        logging.error(f"Error replicando en {nodo}: {e}")


- replicar_archivo_async: Envía los datos cifrados a otro nodo de manera asíncrona.*texto en cursiva*

**Función para Replicar Archivos**

Función para iniciar la replicación de archivos.

In [ ]:
def replicar_archivo(nodo, ruta_archivo, nonce, tag):
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(replicar_archivo_async(nodo, ruta_archivo, nonce, tag))
    loop.close()


- replicar_archivo: Inicia el evento asíncrono para replicar archivos.

**Función de Carga de Archivos**

Función para cargar archivos en el sistema, cifrarlos y replicarlos a otros nodos.

In [ ]:
@app.route('/cargar', methods=['POST'])
def cargar_archivo():
    archivo = request.files['archivo']
    ruta_archivo = os.path.join(UPLOAD_FOLDER, archivo.filename)

    try:
        # Cifrar el archivo
        nonce, tag, ciphertext = cifrar_archivo(archivo.read())

        # Guardar el archivo cifrado
        with open(ruta_archivo, 'wb') as file_enc:
            file_enc.write(nonce)
            file_enc.write(tag)
            file_enc.write(ciphertext)

        # Replicar el archivo en otros nodos
        for nodo in NODOS:
            queue.put((nodo, ruta_archivo, nonce, tag))

        logging.debug(f"Archivo {archivo.filename} cargado y cifrado exitosamente")
        return 'Archivo cargado y cifrado exitosamente', 200
    except Exception as e:
        logging.error(f"Error al cargar y cifrar el archivo: {e}")
        return jsonify({'error': f'Error al cargar y cifrar el archivo: {e}'}), 500


- cargar_archivo: Cifra el archivo recibido y lo guarda en el sistema, luego inicia la replicación a otros nodos.

**Función de Descarga de Archivos**

Función para descargar y descifrar archivos del sistema.

In [ ]:
@app.route('/descargar/<nombre_archivo>', methods=['GET'])
def descargar_archivo(nombre_archivo):
    try:
        ruta_archivo = os.path.join(UPLOAD_FOLDER, nombre_archivo)
        with open(ruta_archivo, 'rb') as file_enc:
            nonce = file_enc.read(16)
            tag = file_enc.read(16)
            ciphertext = file_enc.read()

        data = descifrar_archivo(nonce, tag, ciphertext)
        temp_file_path = os.path.join(UPLOAD_FOLDER, f"temp_{nombre_archivo}")
        with open(temp_file_path, 'wb') as temp_file:
            temp_file.write(data)

        return send_file(temp_file_path, as_attachment=True, download_name=nombre_archivo)
    except Exception as e:
        logging.error(f"Error al descargar y descifrar el archivo: {e}")
        return jsonify({'error': f'Error al descargar y descifrar el archivo: {e}'}), 500


   -  descargar_archivo: Descifra y envía el archivo solicitado al usuario.



**Función de Eliminación de Archivos**

Función para eliminar archivos del sistema.

In [ ]:
@app.route('/eliminar/<nombre_archivo>', methods=['DELETE'])
def eliminar_archivo(nombre_archivo):
    ruta_archivo = os.path.join(UPLOAD_FOLDER, nombre_archivo)
    try:
        os.remove(ruta_archivo)
        logging.info(f"Archivo {nombre_archivo} eliminado exitosamente")
        return 'Archivo eliminado exitosamente', 200
    except Exception as e:
        logging.error(f"Error al eliminar el archivo: {e}")
        return jsonify({'error': 'Error al eliminar el archivo'}), 500


- **eliminar_archivo**: Elimina el archivo especificado del sistema.

**Función Worker**

Función worker para manejar la replicación de archivos en segundo plano.

In [ ]:
def worker():
    while True:
        nodo, ruta_archivo, nonce, tag = queue.get()
        if nodo is None:
            break
        replicar_archivo(nodo, ruta_archivo, nonce, tag)
        queue.task_done()


- worker: Toma archivos de la cola y los replica a otros nodos.

**Configuración de la Cola y los Hilos**

Configuración para iniciar los hilos worker.

In [ ]:
num_worker_threads = 5
queue = Queue()

threads = []
for _ in range(num_worker_threads):
    t = threading.Thread(target=worker)
    t.start()
    threads.append(t)


- num_worker_threads: Número de hilos worker.
- queue: Cola para manejar las tareas de replicación.
- threads: Lista de hilos worker.

**nicio de la Aplicación**

Código para iniciar la aplicación Flask.

In [ ]:
if __name__ == "__main__":
    try:
        app.run(host="0.0.0.0", port=5000, debug=True)
    finally:
        for _ in range(num_worker_threads):
            queue.put((None, None, None, None))
        for t in threads:
            t.join()



- app.run: Inicia la aplicación Flask.
- queue.put: Agrega una tarea nula para cada hilo worker.
- threads.join: Espera a que todos los hilos terminen.

